**notebook für multilevelklassifikation auf iNat19 Datensatz. (Training, Testing, Metriken)**




**ex gibt 2 verschiedene trainingsmethoden. die erste kann nur eine batchsize von 1 haben, ist dafür aber auch weniger aufwendig auszuführen, und eher zum testen des mappings der klassen gedacht. die zweite methode trainiert jedes mlp manuell einzeln mit eine beliebigen batchsize**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!git clone https://github.com/cocodataset/cocoapi

!cd /content/cocoapi/PythonAPI && make

In [ ]:
 !pip install timm
 
 !git clone https://github.com/Moldazien/BA.git

In [4]:
import os
os.chdir('/content/BA')

In [5]:
!cd BA && git pull

/bin/bash: line 0: cd: BA: No such file or directory


In [6]:
!mkdir /content/dataset
!cp -R PATH_TO_FEATURES /content/dataset/features

In [7]:
import os
import sys
import argparse
import cv2
import random
import colorsys
import requests
from io import BytesIO

import skimage.io
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms as pth_transforms
import numpy as np
from PIL import Image

import utils
import vision_transformer as vits

from pycocotools.coco import COCO

In [14]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [8]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

class Seg_Dataset(Dataset):
  def __init__(self, annotations_file, dataset_dir, taxonomy, train):  #taxonomy must be: kingdom phylum class order family genus name
    self.train = train
    self.annotations_file = annotations_file
    self.coco = COCO(annotations_file)  #for fast reading
    self.toplvl = 'genus'######################################################## #muss hier verändert werden, um auf anderer ebene zu testen
    
    self.dataset = dataset_dir
    self.img_ids = self.coco.getImgIds()

    def mapping(taxonomy):
      mapping = {}
      cat_ids = self.coco.getCatIds()
      categories = self.coco.loadCats(cat_ids)
      by_tax_cat = list(set([cat[taxonomy] for cat in categories]))
      by_tax_cat.sort()
      numb_cats = len(by_tax_cat)
      for i in range(numb_cats):
        for cat in categories:
          if cat[taxonomy] == by_tax_cat[i]:
            mapping[cat['id']] = i + 1
      return mapping
      
    self.upcls_mapping = mapping(self.toplvl)
    self.upcls_len = len(self.upcls_mapping)
    self.lower_mapping = mapping('name')
    self.lower_len = len(self.lower_mapping)

    def reverse_mapping(taxonomy, name_mapping):
      gen_to_name = {}
      cat_ids = self.coco.getCatIds()
      categories = self.coco.loadCats(cat_ids)
      by_tax_cat = list(set([cat[taxonomy] for cat in categories]))
      by_tax_cat.sort() 
      numb_cats = len(by_tax_cat)
      for i in range(numb_cats):
        gen_list = []
        for elem in categories:
          if elem[taxonomy] == by_tax_cat[i]:
            gen_list.append(elem['id'])
        gen_to_name[i+1] = gen_list
      for elem in gen_to_name:
        for i in range(len(gen_to_name[elem])):
          gen_to_name[elem][i] = name_mapping[gen_to_name[elem][i]]
      return gen_to_name

    self.name_genus = reverse_mapping(self.toplvl, self.lower_mapping)


  def __len__(self):
    return len(self.img_ids)

  def __getitem__(self, idx):
    img_id = self.img_ids[idx]
    img = self.coco.loadImgs(img_id)
    ann_ids = self.coco.getAnnIds(img[0]['id'])
    anns = self.coco.loadAnns(ann_ids)
    fileN = img[0]['file_name'].split('/')

    classN = fileN[1]
    direcN = fileN[2]
    imgN = fileN[3].split('.')[0]
    feature_path = self.dataset + '/features/' + classN + '/' + direcN + '/' + imgN + '_feature.pt'
    features = torch.load(feature_path, map_location=torch.device('cpu'))
    ground_truth = 0
    gt_genus = 0
    if len(anns) > 0:
      ground_truth = self.lower_mapping[anns[0]['category_id']]
      gt_genus = self.upcls_mapping[anns[0]['category_id']]
    super_cut = self.name_genus[gt_genus]
    gt_vect = np.zeros(len(super_cut))

    for i in range(len(super_cut)):
      if ground_truth == super_cut[i]:
        gt_vect[i] = 1
    gt_vect = torch.tensor(gt_vect)
    gt = None

    if self.train:
      gt = gt_vect
    else:
      gt = ground_truth

    return features, gt, gt_genus 

In [52]:
import os
from torchvision.io import read_image
from torch.utils.data import Dataset, DataLoader

#datenklasse, um aus obiger datenklasse nur elemente zu bekommen, die aus einer bestimmten klasse der höheren Taxonomieebene sind
class PerKlassDataset(Dataset):
  def __init__(self, dataset, trainingsschritt):  #taxonomy must be: kingdom phylum class order family genus name
    self.subdataset = dataset 
    self.step = trainingsschritt    #variable die beschreibt, bei welcher höheren klasse man gerade ist

    self.ann_file = self.subdataset.coco
    self.needed_ids = self.subdataset.name_genus[self.step]

    self.cut_img_ids = []

    self.name_genus = self.subdataset.name_genus

    for id in self.subdataset.img_ids:
      img =self.subdataset.coco.loadImgs(id)
      ann_ids = self.subdataset.coco.getAnnIds(img[0]['id'])
      anns = self.subdataset.coco.loadAnns(ann_ids)
      if len(anns) > 0:
        cat_id = self.subdataset.lower_mapping[anns[0]['category_id']]
        if cat_id in self.needed_ids:
          self.cut_img_ids.append(self.subdataset.img_ids.index(id))


  def __len__(self):
    return len(self.cut_img_ids)

  def __getitem__(self, idx):
    real_id = self.cut_img_ids[idx]
    features, gt, gt_genus = self.subdataset.__getitem__(real_id)

    return features, gt, gt_genus 

In [ ]:
dataset_path = '/content/dataset'

trainset = Seg_Dataset('TRAIN_ANNOTATONS.json', dataset_path, 'name', True) #muss hier immer name sein
testset = Seg_Dataset('TEST_ANNOTATONS.json', dataset_path, 'name', False) 

In [ ]:
print(testset.name_genus)

for i in testset.name_genus:
  print(len(testset.name_genus[i]))

In [15]:
import random
random.seed(7)

train_ids = np.arange(0,trainset.__len__(),1)
test_ids = np.arange(0,testset.__len__(),1)

# model


In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MLP_net(nn.Module):

    def __init__(self, input_size, output_size):
      super(MLP_net, self).__init__()

      self.lin1 = nn.Linear(input_size, input_size, bias=True)
      self.lin2 = nn.Linear(input_size, input_size, bias=True)
      #self.lin2plus = nn.Linear(input_size, input_size, bias=True)
      self.lin3 = nn.Linear(input_size, int(input_size/2), bias=True)
      self.lin3plus = nn.Linear(int(input_size/2), int(input_size/2), bias=True)
      #self.lin3plus1 = nn.Linear(int(input_size/2), int(input_size/2), bias=True)
      self.lin4 = nn.Linear(int(input_size/2), output_size, bias=True)

    def forward(self, x):
      x = self.lin1(x)
      x = F.relu(x)
      x = self.lin2(x)
      x = F.relu(x)
      #x = self.lin2plus(x)
      #x = F.relu(x)
      x = self.lin3(x)
      x = F.relu(x)
      x = self.lin3plus(x)
      x = F.relu(x)
      #x = self.lin3plus1(x)
      #x = F.relu(x)
      x = self.lin4(x)
      #x = F.softmax(x, dim = 0)#F.sigmoid(x)#
      return x

In [ ]:
load_models = False
PATH = 'PATH/MODEL'
network_list = []

#erstellen einer liste von k verschiedenen modellen (14 für gattung)

for i in range(len(trainset.name_genus)):
  print(i)
  net = net = MLP_net(768, len(trainset.name_genus[i+1]))
  net.to(device)

  if load_models:
    full_path = PATH + str(i+1) + '.pth'
    net.load_state_dict(torch.load(full_path))

  net.train()

  network_list.append(net)


#for i in range(len(trainset.name_genus)):
#  print('Saved ' + str(i + 1))
#  full_path = PATH + str(i + 1) + '.pth'

In [ ]:
import torch.optim as optim

#erstellen einer liste von optimizern. je ein optimizer pro mlp in der liste

criterion = nn.CrossEntropyLoss()

optimizer_list = []

for i in range(len(trainset.name_genus)):
  print(i)
  optimizer = optim.Adam(network_list[i].parameters(), lr = 0.001) #0.001
  optimizer_list.append(optimizer)

**naives training mit batchsize von 1. nur zum testen gedacht. hier muss nicht jedes mlp einzeln manuell trainiert werden**

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

dataset_path = '/content/dataset'

trainset = Seg_Dataset('TRAIN_ANNOTATONS.json', dataset_path, 'name', True) #muss hier name sein

batch_size = 1  #geht hier nicht anders

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, mask, gt_genus = data

        #print(gt_genus)
        gt_genus = int(gt_genus[0])

        inputs = inputs.to(device)
        inputs = inputs.reshape(768)

        mask = mask.to(device)
        mask = mask.type(torch.float32)

        # zero the parameter gradients
        optimizer_list[gt_genus-1].zero_grad()

        # forward + backward + optimize

        outputs = network_list[gt_genus-1](inputs)
       
        outputs = outputs.reshape((1, len(trainset.name_genus[gt_genus]))) #keine ahnung

        #print(mask)
        #print(outputs.shape)
        #print(trainset.name_genus[gt_genus])

        loss = criterion(outputs, mask)
        loss.backward()

        optimizer_list[gt_genus-1].step()

        # print statistics
        running_loss += loss.item()

        loss_graph[gt_genus-1].append(loss.item())

        output_value = 2000
        if i % output_value == output_value - 1:    # print every 2000 mini-batches
          print('[%d, %5d] loss: %.20f' %
              (epoch + 1, i + 1, running_loss / output_value))
          
          full_loss_graph.append(running_loss / output_value)

          running_loss = 0.0
          #PATH = '/content/gdrive/MyDrive/models/nets/simple/' + str(i) + 'model_mlp.pth'
          #torch.save(net.state_dict(), PATH)

print('Finished Training')

PATH = 'MODELS_DIRECTORY'

for i in range(len(trainset.name_genus)): #speichern aller einzelnen modelle
  print('Saved ' + str(i + 1))
  full_path = PATH + str(i + 1) + 'genus.pth'
  torch.save(net.state_dict(), full_path)

In [ ]:
N = 500

#gesamtloss
for item in loss_graph:
  loss_arr = np.asarray(item)
  loss_arr = np.convolve(loss_arr, np.ones(N)/N, mode='valid')

  x_val = np.arange(0, loss_arr.size, 1)

  plt.plot(x_val, loss_arr)
  plt.ylabel('loss')
  plt.show()


In [ ]:
import matplotlib.pyplot as plt

#loss per class in höherer taxonomieebene
loss_arr = np.asarray(full_loss_graph)
x_val = np.arange(0, len(full_loss_graph), 1)

plt.plot(x_val, loss_arr)
plt.ylabel('loss')
plt.show()

**manuelles Training für jedes einzelnen mlp mit verschiedenen batchsizes**

**Training muss einmal für alle MLPs einzeln durchgeführt werden (beispielsweise für die Gattung mit 1-14). Danach kann inferenz ausgeführt werden** 

In [ ]:
dataset_path = '/content/dataset'

trainset1 = Seg_Dataset('/content/gdrive/MyDrive/Datasets/iNaturalist2019/annotations/meg_train.json', dataset_path, 'name', True) #class order family genus name

trainset = PerKlassDataset(trainset1, 1)  #startet bei 1. (genus bis 14)

testset = Seg_Dataset('/content/gdrive/MyDrive/Datasets/iNaturalist2019/annotations/meg_test.json', dataset_path, 'name', False)

batch_size = 8

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

In [ ]:
for epoch in range(30):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, mask, gt_genus = data
        gt_genus = int(gt_genus[0])
        inputs = inputs.to(device)
        numb_inst = inputs.shape[0]
        #inputs = inputs.reshape(768)

        mask = mask.to(device)
        mask = mask.type(torch.float32)

        # zero the parameter gradients
        optimizer_list[gt_genus-1].zero_grad()

        # forward + backward + optimize

        outputs = network_list[gt_genus-1](inputs)
       
        outputs = outputs.reshape((numb_inst, len(trainset.name_genus[gt_genus]))) #keine ahnung

        #print(mask)
        #print(outputs.shape)
        #print(trainset.name_genus[gt_genus])

        loss = criterion(outputs, mask)
        loss.backward()

        optimizer_list[gt_genus-1].step()

        running_loss += loss.item()

        #output_value = 2000
        #if i % output_value == output_value - 1:    # print every 2000 mini-batches
        #  print('[%d, %5d] loss: %.20f' %
        #      (epoch + 1, i + 1, running_loss / output_value))
        #  
        #  full_loss_graph.append(running_loss / output_value)
        #
        #  running_loss = 0.0
        #  #PATH = '/content/gdrive/MyDrive/models/nets/simple/' + str(i) + 'model_mlp.pth'
        #  #torch.save(net.state_dict(), PATH)

print('Finished Training')
PATH = 'MODELS_DIRECTORY'
for i in range(len(trainset.name_genus)): #speichern aller einzelnen modelle
  print('Saved ' + str(i + 1))
  full_path = PATH + str(i + 1) + 'genus.pth'
  #torch.save(net.state_dict(), full_path)

**knn-Klassifikator wird hier geladen. falls kein modell vorhanden ist, ist ganz unten im notebook ein bereich um ein knn klassifikator zu trainiern**

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as kNN
from joblib import dump, load

clf = kNN(n_neighbors=20)

clf = load('KNN.joblib') 

**testing auf trainingsdatensatz**


In [63]:
#testing auf testdatensatz. errechnete ergebnisse der mlps werden auf die einzelnen klassen auf speziesebene umgerechnet

Ygt = []
Ypred = []

net.eval()

counter = 0

for i in test_ids:
  features, gt, gt_genus = testset.__getitem__(i)

  net_features = features.reshape(768)
  npfeatures = np.asarray(features)

  knn_pred = clf.predict(npfeatures)[0] #knn prediction
  species = testset.name_genus[knn_pred]

  Ygt.append(gt)

  net_features = net_features.to(device)
  net_pred = network_list[knn_pred-1](net_features)
  net_pred = net_pred.detach()
  net_pred = net_pred.to('cpu')

  net_pred = F.softmax(net_pred, dim = 0)

  pred_array = np.asarray(net_pred, dtype=np.float32)
  
  top = np.amax(pred_array)
  class_pred = int(np.where(pred_array == top)[0][0])
  class_pred = species[class_pred]
  

  Ypred.append(class_pred)
  counter = counter + 1

In [64]:
YclassesGt = Ygt
YclassesPred = Ypred

In [65]:
Ypred = YclassesPred
Ygt = YclassesGt

In [66]:
from sklearn.metrics import confusion_matrix as confusion

matr = confusion(Ygt, Ypred)  #confusionmatrix

In [67]:
norm_matrix = np.zeros(matr.shape)

for i in range(matr.shape[0]):
  for j in range(matr.shape[1]):
    norm_matrix[i,j] = matr[i,j]/sum(matr[i,:])


In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

#confusionmatrizen in schön
disp = ConfusionMatrixDisplay(matr)
disp.plot()

disp1 = ConfusionMatrixDisplay(norm_matrix)
disp1.plot()
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, average_precision_score, f1_score, precision_score, recall_score

accuracy = accuracy_score(Ygt, Ypred)
#avg_prec = average_precision_score(Ygt, Ypred)
f1 = f1_score(Ygt, Ypred, average = 'macro')
precision = precision_score(Ygt, Ypred, average = 'macro')
recall = recall_score(Ygt, Ypred, average = 'macro')

In [ ]:
print(accuracy)
print(f1)
print(precision)
print(recall)

**training für kNN-Klassifikator für oben**

In [ ]:
trainset = Seg_Dataset('TRAIN_ANNOTATONS.json', dataset_path, 'genus', False)

In [58]:
import random
random.seed(7)

train_ids = np.arange(0,trainset.__len__(),1)

In [ ]:
X = []
Y = []


for i in train_ids:
  features, speciec_gt, gt = trainset.__getitem__(i)
  
  print(i)

  features = features.reshape(-1)

  npfeatures = np.asarray(features)
  
  X.append(npfeatures)
  Y.append(gt)

In [ ]:
print(len(X))
print(len(Y))

In [ ]:
X_arr = np.asarray(X)
Y_arr = np.asarray(Y)

print(X_arr.shape)
print(Y_arr.shape)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as kNN

clf = kNN(n_neighbors=20)

X_svm = X_arr
Y_svm = Y_arr

clf.fit(X_svm, Y_svm)